# Plan: PetFinder Pawpularity (medal-oriented)

Objectives:
- Establish strong, reliable CV and fast baseline quickly.
- Progress to high-signal image models + metadata, then blend.
- Always log progress/time per fold; iterate with subsamples first.

Milestones:
1) Environment & Data Checks
   - Verify GPU (nvidia-smi) and install correct cu121 torch/vision if needed.
   - Read train.csv/test.csv; verify image paths; basic stats/target distribution.

2) Validation Protocol
   - 5-fold CV, stratify by Pawpularity bins (e.g., 10-bin or quantiles).
   - Single deterministic split saved to disk; reuse for all models.

3) Baselines
   - Tabular baseline: LightGBM/XGBoost on metadata (the 12 binary features) + simple image stats (width/height/aspect/brightness). Quick 5-fold RMSE.
   - CNN baseline: Pretrained EfficientNet-B0 or ConvNeXt-Tiny at 256–320 px, RMSE loss, simple aug, 5-fold, infer TTA=off initially.

4) Improved Vision Model
   - Upgrade backbone (e.g., Swin-T/ConvNeXt-S/EfficientNetV2-S), 384–448 px, strong aug (RandomResizedCrop, HFlip, ColorJitter, CutMix/MixUp optional).
   - Cosine LR, warmup, EMA, label smoothing, early stopping by OOF.
   - Use per-fold checkpointing; predict averaged epochs near best.

5) Metadata Fusion
   - Late-fusion head: concat CNN pooled embedding + normalized metadata; small MLP head.
   - Compare vs training CNN on images only and blending with tabular model.

6) Ensembling
   - Blend diverse backbones/resolutions/seeds; weight by OOF.
   - Keep TTA modest (e.g., 4 flips) if helps.

7) Error Analysis
   - Bucket by target bins and brightness/aspect; adjust aug/resolution/center-crop policy.

Targets:
- Baseline tabular: RMSE ~19–20
- Baseline CNN small: ~17.5–17.8
- Upgraded model + fusion/ensemble: ≤17.1 (bronze) with aim for ≤17.0 (silver).

Next Steps:
- Expert review on plan.
- Add GPU/env check cell; install PyTorch cu121 if needed.
- Implement CV splitter and quick tabular baseline (with simple image stats).
- Implement CNN baseline script with logging; run 5-fold at 256–320 px.
- Iterate toward fusion/ensembling.

In [1]:
# Environment and data sanity checks
import os, sys, json, time, subprocess, pandas as pd

def run(cmd):
    print('$', ' '.join(cmd), flush=True)
    return subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True).stdout

print('Python:', sys.version)
print('CWD:', os.getcwd())

# 1) GPU check
print('--- nvidia-smi ---')
print(run(['bash','-lc','nvidia-smi || true']))

# 2) Dataset presence and counts
train_csv = 'train.csv'
test_csv = 'test.csv'
train_dir = 'train'
test_dir = 'test'
assert os.path.exists(train_csv) and os.path.exists(test_csv), 'CSV files missing'
assert os.path.isdir(train_dir) and os.path.isdir(test_dir), 'Image directories missing'

df_train = pd.read_csv(train_csv)
df_test = pd.read_csv(test_csv)
print('train.csv shape:', df_train.shape)
print('test.csv shape:', df_test.shape)
print('train columns:', list(df_train.columns))
print('test columns:', list(df_test.columns))

# Verify image files exist for a quick sample
missing_train = 0
for i, row in df_train.head(50).iterrows():
    fp = os.path.join(train_dir, f"{row['Id']}.jpg")
    if not os.path.exists(fp):
        missing_train += 1
missing_test = 0
for i, row in df_test.head(50).iterrows():
    fp = os.path.join(test_dir, f"{row['Id']}.jpg")
    if not os.path.exists(fp):
        missing_test += 1
print('Sample check missing files -> train:', missing_train, 'test:', missing_test)

# Basic target stats
if 'Pawpularity' in df_train.columns:
    print('Pawpularity describe (first look):')
    print(df_train['Pawpularity'].describe())

print('Setup OK. Next: lock CV splits and install PyTorch cu121 stack if needed.')

Python: 3.11.0rc1 (main, Aug 12 2022, 10:02:14) [GCC 11.2.0]
CWD: /var/lib/simon/agent_run_states/petfinder-pawpularity-score-20250927-162350
--- nvidia-smi ---
$ bash -lc nvidia-smi || true


Sat Sep 27 16:32:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.06             Driver Version: 550.144.06     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q                 On  |   00000002:00:00.0 Off |                    0 |
| N/A   N/A    P0             N/A /  N/A  |     182MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Install exact cu121 PyTorch stack and verify GPU
import os, sys, shutil, subprocess
from pathlib import Path

def pip(*args):
    print('>', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

# 0) Uninstall any existing torch stacks (ignore errors)
for pkg in ('torch','torchvision','torchaudio'):
    subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', pkg], check=False)

# Clean possible stray site dirs that can shadow correct wheels (idempotent)
for d in (
    '/app/.pip-target/torch',
    '/app/.pip-target/torch-2.8.0.dist-info',
    '/app/.pip-target/torch-2.4.1.dist-info',
    '/app/.pip-target/torchvision',
    '/app/.pip-target/torchvision-0.23.0.dist-info',
    '/app/.pip-target/torchvision-0.19.1.dist-info',
    '/app/.pip-target/torchaudio',
    '/app/.pip-target/torchaudio-2.8.0.dist-info',
    '/app/.pip-target/torchaudio-2.4.1.dist-info',
    '/app/.pip-target/torchgen',
    '/app/.pip-target/functorch',
):
    if os.path.exists(d):
        print('Removing', d)
        shutil.rmtree(d, ignore_errors=True)

# 1) Install the EXACT cu121 torch stack
pip('install',
    '--index-url', 'https://download.pytorch.org/whl/cu121',
    '--extra-index-url', 'https://pypi.org/simple',
    'torch==2.4.1', 'torchvision==0.19.1', 'torchaudio==2.4.1')

# 2) Freeze versions for later installs
Path('constraints.txt').write_text(
    'torch==2.4.1\n'
    'torchvision==0.19.1\n'
    'torchaudio==2.4.1\n'
)

# 3) Sanity check
import torch
print('torch:', torch.__version__, 'built CUDA:', getattr(torch.version, 'cuda', None))
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU:', torch.cuda.get_device_name(0))
assert str(getattr(torch.version,'cuda','')).startswith('12.1'), f'Wrong CUDA build: {torch.version.cuda}'
assert torch.cuda.is_available(), 'CUDA not available'
print('PyTorch cu121 stack installed and verified.')

> install --index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://pypi.org/simple torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1


Looking in indexes: https://download.pytorch.org/whl/cu121, https://pypi.org/simple


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 MB 265.5 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 269.7 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 126.9 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 251.3 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 252.1 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 254.1 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 KB 500.4 MB/s eta 0:00:00


torch: 2.4.1+cu121 built CUDA: 12.1
CUDA available: True
GPU: NVIDIA A10-24Q
PyTorch cu121 stack installed and verified.


In [3]:
# Create and save 5-fold stratified CV splits (qcut=10 bins)
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold

SEED = 42
N_FOLDS = 5
N_BINS = 10

df = pd.read_csv('train.csv')

# Create stratification bins on target
bins = pd.qcut(df['Pawpularity'], q=N_BINS, labels=False, duplicates='drop')
df['bin'] = bins

skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
folds = np.full(len(df), -1, dtype=int)

for fold, (trn_idx, val_idx) in enumerate(skf.split(df, df['bin'])):
    folds[val_idx] = fold

df['fold'] = folds
assert (df['fold']>=0).all(), 'Unassigned fold rows exist'

# Save splits to disk for reuse
cols_to_save = ['Id','Pawpularity','fold'] + [c for c in df.columns if c not in ['Id','Pawpularity','fold','bin']]
df[cols_to_save].to_csv('train_folds.csv', index=False)
df[['Id','fold']].to_csv('folds.csv', index=False)

# Log fold sizes and target means
print('Saved train_folds.csv and folds.csv')
for f in range(N_FOLDS):
    m = df.loc[df['fold']==f, 'Pawpularity'].mean()
    n = (df['fold']==f).sum()
    print(f'Fold {f}: n={n}, mean Pawpularity={m:.3f}')

# Clean up helper column
df.drop(columns=['bin'], inplace=True)

Saved train_folds.csv and folds.csv
Fold 0: n=1784, mean Pawpularity=38.078
Fold 1: n=1784, mean Pawpularity=38.007
Fold 2: n=1784, mean Pawpularity=38.016
Fold 3: n=1784, mean Pawpularity=38.105
Fold 4: n=1784, mean Pawpularity=37.917


In [4]:
# Tabular baseline: metadata + simple image stats with XGBoost (OOF RMSE + submission)
import os, sys, time, math, json, shutil, subprocess, gc
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from PIL import Image, ImageOps

def ensure_pkg(pkg, pip_name=None):
    pip_name = pip_name or pkg
    try:
        __import__(pkg)
    except ImportError:
        print(f'Installing {pip_name} ...', flush=True)
        subprocess.run([sys.executable, '-m', 'pip', 'install', '-c', 'constraints.txt', pip_name, '--upgrade-strategy', 'only-if-needed'], check=True)

ensure_pkg('xgboost')
ensure_pkg('cv2', 'opencv-python-headless')
from tqdm import tqdm
import xgboost as xgb
import cv2

t0 = time.time()
train_dir, test_dir = 'train', 'test'
df_train = pd.read_csv('train_folds.csv')  # contains fold column
df_test = pd.read_csv('test.csv')

# Compute/cached simple image stats
stats_path_train = Path('img_stats_train.parquet')
stats_path_test = Path('img_stats_test.parquet')

def compute_stats(df, img_dir, out_path):
    if out_path.exists():
        return pd.read_parquet(out_path)
    rows = []
    for _, r in tqdm(df.iterrows(), total=len(df), desc=f'img-stats {img_dir}'):
        img_fp = os.path.join(img_dir, f"{r['Id']}.jpg")
        try:
            img = Image.open(img_fp)
            img = ImageOps.exif_transpose(img)
            w, h = img.size
            arr = np.asarray(img.convert('RGB'))
            gray = cv2.cvtColor(arr, cv2.COLOR_RGB2GRAY)
            mean = float(gray.mean())
            std = float(gray.std())
            lap = cv2.Laplacian(gray, cv2.CV_64F)
            sharp = float(lap.var())
            rows.append({'Id': r['Id'], 'width': w, 'height': h, 'aspect': w/(h+1e-6), 'brightness_mean': mean, 'brightness_std': std, 'sharpness': sharp})
        except Exception as e:
            rows.append({'Id': r['Id'], 'width': np.nan, 'height': np.nan, 'aspect': np.nan, 'brightness_mean': np.nan, 'brightness_std': np.nan, 'sharpness': np.nan})
    out = pd.DataFrame(rows)
    out.to_parquet(out_path, index=False)
    return out

stats_tr = compute_stats(df_train, train_dir, stats_path_train)
stats_te = compute_stats(df_test, test_dir, stats_path_test)

# Merge features
meta_cols = ['Subject Focus','Eyes','Face','Near','Action','Accessory','Group','Collage','Human','Occlusion','Info','Blur']
feat_cols = meta_cols + ['width','height','aspect','brightness_mean','brightness_std','sharpness']
df_tr = df_train.merge(stats_tr, on='Id', how='left')
df_te = df_test.merge(stats_te, on='Id', how='left')

# Fill NaNs and scale numeric (image stats); keep binaries as-is
num_cols = ['width','height','aspect','brightness_mean','brightness_std','sharpness']
scaler = StandardScaler()
df_tr[num_cols] = scaler.fit_transform(df_tr[num_cols])
df_te[num_cols] = scaler.transform(df_te[num_cols])

X = df_tr[feat_cols].astype(float).values
y = df_tr['Pawpularity'].values.astype(float)
folds = df_tr['fold'].values
X_test = df_te[feat_cols].astype(float).values

# XGBoost params
use_gpu = True
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'learning_rate': 0.03,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'lambda': 1.0,
    'tree_method': 'gpu_hist' if use_gpu else 'hist',
}

oof = np.zeros(len(df_tr), dtype=float)
pred_test = np.zeros(len(df_te), dtype=float)
nrounds = 2000
esr = 100

for f in range(5):
    tr_idx = np.where(folds != f)[0]
    va_idx = np.where(folds == f)[0]
    dtr = xgb.DMatrix(X[tr_idx], label=y[tr_idx])
    dva = xgb.DMatrix(X[va_idx], label=y[va_idx])
    dte = xgb.DMatrix(X_test)
    start = time.time()
    print(f'[Fold {f}] train={len(tr_idx)} val={len(va_idx)}', flush=True)
    booster = xgb.train(params, dtr, num_boost_round=nrounds, evals=[(dtr,'train'),(dva,'valid')], early_stopping_rounds=esr, verbose_eval=100)
    oof[va_idx] = booster.predict(dva)
    pred_test += booster.predict(dte) / 5.0
    elapsed = time.time() - start
    rmse = mean_squared_error(y[va_idx], oof[va_idx], squared=False)
    print(f'[Fold {f}] RMSE={rmse:.4f} elapsed={elapsed:.1f}s', flush=True)
    del dtr, dva, dte, booster; gc.collect()

oof_rmse = mean_squared_error(y, oof, squared=False)
print(f'OOF RMSE: {oof_rmse:.4f}')

# Clip predictions and save artifacts
oof_clip = np.clip(oof, 0, 100)
pred_test_clip = np.clip(pred_test, 0, 100)
pd.DataFrame({'Id': df_tr['Id'], 'oof': oof_clip, 'target': y, 'fold': folds}).to_csv('oof_tabular_xgb.csv', index=False)
sub = pd.DataFrame({'Id': df_te['Id'], 'Pawpularity': pred_test_clip})
sub.to_csv('submission.csv', index=False)
print('Saved submission.csv, shape:', sub.shape, 'head:', sub.head().to_dict('records'))
print('Total time:', f'{time.time()-t0:.1f}s')

Installing opencv-python-headless ...


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 230.9 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 164.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 286.7 MB/s eta 0:00:00


img-stats train:   0%|          | 0/8920 [00:00<?, ?it/s]

img-stats train:   0%|          | 13/8920 [00:00<01:13, 121.05it/s]

...

[100]	train-rmse:19.15359	valid-rmse:20.34200


[118]	train-rmse:18.94411	valid-rmse:20.36454


[Fold 4] RMSE=20.3645 elapsed=0.3s


OOF RMSE: 20.6720
Saved submission.csv, shape: (992, 2) head: [{'Id': 'ee51b99832f1ba868f646df93d2b6b81', 'Pawpularity': 37.957308769226074}, {'Id': 'caddfb3f8bff9c4b95dbe022018eea21', 'Pawpularity': 38.96543836593628}, {'Id': '582eeabd4a448a53ebb79995888a4b0b', 'Pawpularity': 37.32875967025757}, {'Id': 'afc1ad7f0c5eea880759d09e77f7deee', 'Pawpularity': 38.45194959640503}, {'Id': 'd5bdf3446e86ce4ec67ce7a00f1cccc2', 'Pawpularity': 37.12950944900513}]
Total time: 74.6s


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [6]:
# Install CNN deps (pinned, honoring torch constraints) without touching torch stack
import sys, subprocess

def pip_install_no_deps(pkgs):
    cmd = [sys.executable, '-m', 'pip', 'install', '-c', 'constraints.txt', '--upgrade-strategy', 'only-if-needed', '--no-deps'] + pkgs
    print('>', ' '.join(cmd), flush=True)
    subprocess.run(cmd, check=True)

# Install timm and albumentations without pulling torch/torchvision again
pip_install_no_deps(['timm==1.0.9', 'albumentations==1.4.14'])
print('Installed timm and albumentations (no-deps). Next: implement train_cnn.py and run a 1-fold smoke.')

> /usr/bin/python3.11 -m pip install -c constraints.txt --upgrade-strategy only-if-needed --no-deps timm==1.0.9 albumentations==1.4.14


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 KB 323.2 MB/s eta 0:00:00


Installed timm and albumentations (no-deps). Next: implement train_cnn.py and run a 1-fold smoke.


In [7]:
# Install missing deps for albumentations
import sys, subprocess

def pip_install(pkgs):
    cmd = [sys.executable, '-m', 'pip', 'install', '-c', 'constraints.txt', '--upgrade-strategy', 'only-if-needed'] + pkgs
    print('>', ' '.join(cmd), flush=True)
    subprocess.run(cmd, check=True)

pip_install(['albucore==0.0.33', 'imageio==2.37.0', 'scikit-image==0.25.2'])
print('Installed albucore, imageio, scikit-image.')

> /usr/bin/python3.11 -m pip install -c constraints.txt --upgrade-strategy only-if-needed albucore==0.0.33 imageio==2.37.0 scikit-image==0.25.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 KB 14.6 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 275.1 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 133.8 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.5/496.5 KB 493.3 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 244.9 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 KB 406.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 552.3 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 233.8 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
timm 1.0.9 requires huggingface_hub, which is not installed.
timm 1.0.9 requires safetensors, which is not installed.
albumentations 1.4.14 requires eval-type-backport, which is not installed.


Installed albucore, imageio, scikit-image.


In [13]:
# Run 1-fold 1-epoch smoke for CNN pipeline
import sys, subprocess, time

cmd = [sys.executable, 'train_cnn.py',
       '--model', 'tf_efficientnetv2_s_in21k',
       '--img_size', '384', '--batch_size', '32',
       '--use_fusion', '--tta_hflip', '--smoke', '--fold', '0',
       '--num_workers', '4']
print('Launching:', ' '.join(cmd), flush=True)
t0 = time.time()
proc = subprocess.run(cmd, check=False)
print('Exit code:', proc.returncode, 'Elapsed:', f'{time.time()-t0:.1f}s', flush=True)
assert proc.returncode == 0, 'train_cnn.py smoke run failed'

Launching: /usr/bin/python3.11 train_cnn.py --model tf_efficientnetv2_s_in21k --img_size 384 --batch_size 32 --use_fusion --tta_hflip --smoke --fold 0 --num_workers 4


/app/.pip-target/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnetv2_s_in21k to current tf_efficientnetv2_s.in21k.
  model = create_fn(


/app/.pip-target/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnetv2_s_in21k to current tf_efficientnetv2_s.in21k.
  model = create_fn(


/var/lib/simon/agent_run_states/petfinder-pawpularity-score-20250927-162350/train_cnn.py:319: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device=='cuda'))


Device: cuda
Fold 0 | train=7136 val=1784 epochs=1


...

Epoch 1/1 | loss=0.25296 | val_rmse=32.9759 | lr=[3e-05, 0.001] | time=47.1s
Fold 0 best RMSE: 32.9759 | elapsed 0.8 min | ckpt: work_cnn/tf_efficientnetv2_s_in21k_fold0_best.pth


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Saved OOF to work_cnn/oof_tf_efficientnetv2_s_in21k_fusion1_sz384.csv OOF RMSE: 41.32097792064362
Saved test predictions to work_cnn/submission_tf_efficientnetv2_s_in21k_fusion1_sz384.csv shape: (992, 2)


Exit code: 0 Elapsed: 64.8s


In [12]:
# Install missing deps required by timm pretrained weights (HF hub + safetensors)
import sys, subprocess

def pip_install(pkgs):
    cmd = [sys.executable, '-m', 'pip', 'install', '-c', 'constraints.txt', '--upgrade-strategy', 'only-if-needed'] + pkgs
    print('>', ' '.join(cmd), flush=True)
    subprocess.run(cmd, check=True)

pip_install(['huggingface_hub==0.24.6', 'safetensors==0.4.5'])
print('Installed huggingface_hub and safetensors.')

> /usr/bin/python3.11 -m pip install -c constraints.txt --upgrade-strategy only-if-needed huggingface_hub==0.24.6 safetensors==0.4.5


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 KB 17.9 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 KB 101.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 KB 388.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 KB 425.8 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 KB 315.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 115.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.6/806.6 KB 134.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 KB 433.2 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 KB 497.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 KB 467.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 KB 464.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 KB 388.3 MB/s eta 0:00:00


Installed huggingface_hub and safetensors.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.4.14 requires eval-type-backport, which is not installed.


In [14]:
# Full 5-fold training: EfficientNetV2-S (384px) with fusion, EMA, AMP
import sys, subprocess, time

cmd = [sys.executable, 'train_cnn.py',
       '--model', 'tf_efficientnetv2_s_in21k',
       '--img_size', '384',
       '--batch_size', '32',
       '--epochs', '12',
       '--warmup_epochs', '1',
       '--ema_decay', '0.999',
       '--lr_backbone', '3e-5',
       '--lr_head', '1e-3',
       '--weight_decay', '1e-4',
       '--num_workers', '4',
       '--seed', '42',
       '--tta_hflip',
       '--use_fusion'
]
print('Launching full run:', ' '.join(cmd), flush=True)
t0 = time.time()
proc = subprocess.run(cmd, check=False)
print('Exit code:', proc.returncode, 'Elapsed:', f'{(time.time()-t0)/60:.1f} min', flush=True)
assert proc.returncode == 0, 'EffNetV2-S full run failed'

Launching full run: /usr/bin/python3.11 train_cnn.py --model tf_efficientnetv2_s_in21k --img_size 384 --batch_size 32 --epochs 12 --warmup_epochs 1 --ema_decay 0.999 --lr_backbone 3e-5 --lr_head 1e-3 --weight_decay 1e-4 --num_workers 4 --seed 42 --tta_hflip --use_fusion


/app/.pip-target/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnetv2_s_in21k to current tf_efficientnetv2_s.in21k.
  model = create_fn(


/app/.pip-target/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnetv2_s_in21k to current tf_efficientnetv2_s.in21k.
  model = create_fn(


/var/lib/simon/agent_run_states/petfinder-pawpularity-score-20250927-162350/train_cnn.py:319: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device=='cuda'))


Device: cuda
Fold 0 | train=7136 val=1784 epochs=12


...

Epoch 12/12 | loss=0.00426 | val_rmse=18.9809 | lr=[0.0, 0.0] | time=47.7s
Fold 4 best RMSE: 18.9674 | elapsed 9.6 min | ckpt: work_cnn/tf_efficientnetv2_s_in21k_fold4_best.pth


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Saved OOF to work_cnn/oof_tf_efficientnetv2_s_in21k_fusion1_sz384.csv OOF RMSE: 19.231653782965047
Saved test predictions to work_cnn/submission_tf_efficientnetv2_s_in21k_fusion1_sz384.csv shape: (992, 2)


Exit code: 0 Elapsed: 48.7 min


In [15]:
# Full 5-fold training: ConvNeXt-Small (384px) with fusion, EMA, AMP
import sys, subprocess, time

cmd = [sys.executable, 'train_cnn.py',
       '--model', 'convnext_small.in12k_ft_in1k',
       '--img_size', '384',
       '--batch_size', '32',  # drop to 24 if OOM
       '--epochs', '12',
       '--warmup_epochs', '1',
       '--ema_decay', '0.999',
       '--lr_backbone', '3e-5',
       '--lr_head', '1e-3',
       '--weight_decay', '1e-4',
       '--num_workers', '4',
       '--seed', '42',
       '--tta_hflip',
       '--use_fusion'
]
print('Launching full run:', ' '.join(cmd), flush=True)
t0 = time.time()
proc = subprocess.run(cmd, check=False)
print('Exit code:', proc.returncode, 'Elapsed:', f'{(time.time()-t0)/60:.1f} min', flush=True)
assert proc.returncode == 0, 'ConvNeXt-S full run failed'

Launching full run: /usr/bin/python3.11 train_cnn.py --model convnext_small.in12k_ft_in1k --img_size 384 --batch_size 32 --epochs 12 --warmup_epochs 1 --ema_decay 0.999 --lr_backbone 3e-5 --lr_head 1e-3 --weight_decay 1e-4 --num_workers 4 --seed 42 --tta_hflip --use_fusion


/var/lib/simon/agent_run_states/petfinder-pawpularity-score-20250927-162350/train_cnn.py:319: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device=='cuda'))


Device: cuda
Fold 0 | train=7136 val=1784 epochs=12


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Epoch 1/12 | loss=0.21422 | val_rmse=34.9970 | lr=[3e-05, 0.001] | time=71.6s


...

Epoch 12/12 | loss=0.00085 | val_rmse=18.0262 | lr=[0.0, 0.0] | time=71.8s
Fold 4 best RMSE: 17.9405 | elapsed 14.4 min | ckpt: work_cnn/convnext_small.in12k_ft_in1k_fold4_best.pth


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Saved OOF to work_cnn/oof_convnext_small.in12k_ft_in1k_fusion1_sz384.csv OOF RMSE: 18.21609435749985
Saved test predictions to work_cnn/submission_convnext_small.in12k_ft_in1k_fusion1_sz384.csv shape: (992, 2)


Exit code: 0 Elapsed: 73.2 min


In [19]:
# Ensemble OOF-weighted blend of available CNN models in work_cnn/
import os, glob, pandas as pd, numpy as np
from sklearn.metrics import mean_squared_error

workdir = 'work_cnn'
oof_paths = sorted(glob.glob(os.path.join(workdir, 'oof_*.csv')))
sub_paths = sorted(glob.glob(os.path.join(workdir, 'submission_*.csv')))
print('Found OOF files:', oof_paths)
print('Found SUB files:', sub_paths)
assert len(oof_paths) > 0 and len(sub_paths) > 0, 'No model artifacts to blend yet.'

# Map tag -> oof and sub
def tag_from(path):
    base = os.path.basename(path)
    return base.replace('oof_', '').replace('submission_', '')

oofs = {}
subs = {}
for p in oof_paths:
    tag = tag_from(p)
    df = pd.read_csv(p)
    oofs[tag] = df
for p in sub_paths:
    tag = tag_from(p)
    df = pd.read_csv(p)
    subs[tag] = df

# Keep only tags that have both
tags = [t for t in oofs.keys() if t in subs]
assert len(tags) > 0, 'No matching OOF+SUB pairs'
print('Blend tags:', tags)

# Compute OOF RMSE per tag and weights ~ 1/RMSE^2
rmses = {}
for t in tags:
    d = oofs[t]
    rmse = mean_squared_error(d['target'], d['oof'], squared=False)
    rmses[t] = rmse
weights = {t: 1.0/(rmses[t]**2 + 1e-9) for t in tags}
w_sum = sum(weights.values())
weights = {t: w/w_sum for t, w in weights.items()}
print('OOF RMSEs:', rmses)
print('Weights:', weights)

# Blend test submissions
blend = None
for t in tags:
    df = subs[t].copy()
    df['Pawpularity'] = df['Pawpularity'].clip(0, 100)
    if blend is None:
        blend = df.copy()
        blend.rename(columns={'Pawpularity': f'pred_{t}'}, inplace=True)
    else:
        blend = blend.merge(df.rename(columns={'Pawpularity': f'pred_{t}'}), on='Id', how='inner')

pred_cols = [c for c in blend.columns if c.startswith('pred_')]
pred = np.zeros(len(blend), dtype=float)
for t in tags:
    pred += weights[t] * blend[f'pred_{t}'].values
final_sub = pd.DataFrame({'Id': blend['Id'], 'Pawpularity': np.clip(pred, 0, 100)})
final_sub.to_csv('submission.csv', index=False)
print('Blended submission saved to submission.csv, shape:', final_sub.shape)
final_sub.head()

Found OOF files: ['work_cnn/oof_convnext_small.in12k_ft_in1k_fusion1_sz384.csv', 'work_cnn/oof_convnext_tiny_fusion1_sz384.csv', 'work_cnn/oof_tf_efficientnetv2_s_in21k_fusion1_sz384.csv']
Found SUB files: ['work_cnn/submission_convnext_small.in12k_ft_in1k_fusion1_sz384.csv', 'work_cnn/submission_convnext_tiny_fusion1_sz384.csv', 'work_cnn/submission_tf_efficientnetv2_s_in21k_fusion1_sz384.csv']
Blend tags: ['convnext_small.in12k_ft_in1k_fusion1_sz384.csv', 'convnext_tiny_fusion1_sz384.csv', 'tf_efficientnetv2_s_in21k_fusion1_sz384.csv']
OOF RMSEs: {'convnext_small.in12k_ft_in1k_fusion1_sz384.csv': 18.21609436551175, 'convnext_tiny_fusion1_sz384.csv': 18.144526453713713, 'tf_efficientnetv2_s_in21k_fusion1_sz384.csv': 19.23165377310838}
Weights: {'convnext_small.in12k_ft_in1k_fusion1_sz384.csv': 0.34422466711539135, 'convnext_tiny_fusion1_sz384.csv': 0.34694549047460777, 'tf_efficientnetv2_s_in21k_fusion1_sz384.csv': 0.3088298424100009}
Blended submission saved to submission.csv, shape:

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


,Id,Pawpularity
0,ee51b99832f1ba868f646df93d2b6b81,55.062599
1,caddfb3f8bff9c4b95dbe022018eea21,45.068822
2,582eeabd4a448a53ebb79995888a4b0b,35.200632
3,afc1ad7f0c5eea880759d09e77f7deee,26.694977
4,d5bdf3446e86ce4ec67ce7a00f1cccc2,20.958674


In [18]:
# Full 5-fold training: ConvNeXt-Tiny (in1k, 384px) with fusion, EMA, AMP
import sys, subprocess, time

cmd = [sys.executable, 'train_cnn.py',
       '--model', 'convnext_tiny',
       '--img_size', '384',
       '--batch_size', '32',  # drop to 24 if OOM
       '--epochs', '12',
       '--warmup_epochs', '1',
       '--ema_decay', '0.999',
       '--lr_backbone', '3e-5',
       '--lr_head', '1e-3',
       '--weight_decay', '1e-4',
       '--num_workers', '4',
       '--seed', '42',
       '--tta_hflip',
       '--use_fusion'
]
print('Launching full run:', ' '.join(cmd), flush=True)
t0 = time.time()
proc = subprocess.run(cmd, check=False)
print('Exit code:', proc.returncode, 'Elapsed:', f'{(time.time()-t0)/60:.1f} min', flush=True)
assert proc.returncode == 0, 'ConvNeXt-Tiny full run failed'

Launching full run: /usr/bin/python3.11 train_cnn.py --model convnext_tiny --img_size 384 --batch_size 32 --epochs 12 --warmup_epochs 1 --ema_decay 0.999 --lr_backbone 3e-5 --lr_head 1e-3 --weight_decay 1e-4 --num_workers 4 --seed 42 --tta_hflip --use_fusion


/var/lib/simon/agent_run_states/petfinder-pawpularity-score-20250927-162350/train_cnn.py:319: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device=='cuda'))


Device: cuda
Fold 0 | train=7136 val=1784 epochs=12


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Epoch 1/12 | loss=0.21194 | val_rmse=23.1661 | lr=[3e-05, 0.001] | time=46.0s


...

Epoch 12/12 | loss=0.00117 | val_rmse=18.0919 | lr=[0.0, 0.0] | time=46.9s
Fold 4 best RMSE: 17.7564 | elapsed 9.3 min | ckpt: work_cnn/convnext_tiny_fold4_best.pth


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Saved OOF to work_cnn/oof_convnext_tiny_fusion1_sz384.csv OOF RMSE: 18.144526442690495
Saved test predictions to work_cnn/submission_convnext_tiny_fusion1_sz384.csv shape: (992, 2)


Exit code: 0 Elapsed: 47.6 min


In [20]:
# Smoke test: convnext_small seed=43 with CutMix (fold 0, 3 epochs) and wd=0.05
import sys, subprocess, time

cmd = [sys.executable, 'train_cnn.py',
       '--model', 'convnext_small.in12k_ft_in1k',
       '--img_size', '384',
       '--batch_size', '32',
       '--epochs', '3',
       '--warmup_epochs', '1',
       '--ema_decay', '0.999',
       '--lr_backbone', '3e-5',
       '--lr_head', '1e-3',
       '--weight_decay', '0.05',
       '--num_workers', '4',
       '--seed', '43',
       '--tta_hflip',
       '--use_fusion',
       '--use_cutmix',
       '--cutmix_alpha', '0.5',
       '--cutmix_p', '0.5',
       '--fold', '0'
]
print('Launching smoke run:', ' '.join(cmd), flush=True)
t0 = time.time()
proc = subprocess.run(cmd, check=False)
print('Exit code:', proc.returncode, 'Elapsed:', f'{(time.time()-t0):.1f}s', flush=True)
assert proc.returncode == 0, 'Smoke run failed; check logs.'

Launching smoke run: /usr/bin/python3.11 train_cnn.py --model convnext_small.in12k_ft_in1k --img_size 384 --batch_size 32 --epochs 3 --warmup_epochs 1 --ema_decay 0.999 --lr_backbone 3e-5 --lr_head 1e-3 --weight_decay 0.05 --num_workers 4 --seed 43 --tta_hflip --use_fusion --use_cutmix --cutmix_alpha 0.5 --cutmix_p 0.5 --fold 0


/var/lib/simon/agent_run_states/petfinder-pawpularity-score-20250927-162350/train_cnn.py:354: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device=='cuda'))


Device: cuda
Fold 0 | train=7136 val=1784 epochs=3


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Epoch 1/3 | loss=0.19000 | val_rmse=26.0774 | lr=[3e-05, 0.001] | time=70.2s


...

Epoch 3/3 | loss=0.02840 | val_rmse=18.6981 | lr=[0.0, 0.0] | time=70.4s
Fold 0 best RMSE: 18.6981 | elapsed 3.5 min | ckpt: work_cnn/convnext_small.in12k_ft_in1k_seed43_fold0_best.pth


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Saved OOF to work_cnn/oof_convnext_small.in12k_ft_in1k_fusion1_sz384_seed43.csv OOF RMSE: 39.589261696852404
Saved test predictions to work_cnn/submission_convnext_small.in12k_ft_in1k_fusion1_sz384_seed43.csv shape: (992, 2)


Exit code: 0 Elapsed: 229.7s


In [21]:
# Full 5-fold: convnext_small.in12k_ft_in1k @384, seed=43, CutMix on, wd=0.05
import sys, subprocess, time

cmd = [sys.executable, 'train_cnn.py',
       '--model', 'convnext_small.in12k_ft_in1k',
       '--img_size', '384',
       '--batch_size', '32',
       '--epochs', '12',
       '--warmup_epochs', '1',
       '--ema_decay', '0.999',
       '--lr_backbone', '3e-5',
       '--lr_head', '1e-3',
       '--weight_decay', '0.05',
       '--num_workers', '4',
       '--seed', '43',
       '--tta_hflip',
       '--use_fusion',
       '--use_cutmix',
       '--cutmix_alpha', '0.5',
       '--cutmix_p', '0.5'
]
print('Launching full run:', ' '.join(cmd), flush=True)
t0 = time.time()
proc = subprocess.run(cmd, check=False)
print('Exit code:', proc.returncode, 'Elapsed:', f'{(time.time()-t0)/60:.1f} min', flush=True)
assert proc.returncode == 0, 'ConvNeXt-S seed=43 full run failed'

Launching full run: /usr/bin/python3.11 train_cnn.py --model convnext_small.in12k_ft_in1k --img_size 384 --batch_size 32 --epochs 12 --warmup_epochs 1 --ema_decay 0.999 --lr_backbone 3e-5 --lr_head 1e-3 --weight_decay 0.05 --num_workers 4 --seed 43 --tta_hflip --use_fusion --use_cutmix --cutmix_alpha 0.5 --cutmix_p 0.5


/var/lib/simon/agent_run_states/petfinder-pawpularity-score-20250927-162350/train_cnn.py:354: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device=='cuda'))


Device: cuda
Fold 0 | train=7136 val=1784 epochs=12


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Epoch 1/12 | loss=0.19000 | val_rmse=26.0774 | lr=[3e-05, 0.001] | time=71.2s


...

Epoch 12/12 | loss=0.00207 | val_rmse=17.6309 | lr=[0.0, 0.0] | time=71.7s
Fold 4 best RMSE: 17.6036 | elapsed 14.4 min | ckpt: work_cnn/convnext_small.in12k_ft_in1k_seed43_fold4_best.pth


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Saved OOF to work_cnn/oof_convnext_small.in12k_ft_in1k_fusion1_sz384_seed43.csv OOF RMSE: 17.843361034410087
Saved test predictions to work_cnn/submission_convnext_small.in12k_ft_in1k_fusion1_sz384_seed43.csv shape: (992, 2)


Exit code: 0 Elapsed: 73.1 min


In [35]:
# NNLS-optimized blend of all available models in work_cnn/
import os, glob, pandas as pd, numpy as np
from sklearn.metrics import mean_squared_error
from scipy.optimize import nnls

workdir = 'work_cnn'
oof_paths = sorted(glob.glob(os.path.join(workdir, 'oof_*.csv')))
sub_paths = sorted(glob.glob(os.path.join(workdir, 'submission_*.csv')))
print('Found OOF files:', len(oof_paths))
print('Found SUB files:', len(sub_paths))
assert len(oof_paths) > 0 and len(sub_paths) > 0, 'No model artifacts to blend yet.'

def tag_from(path):
    base = os.path.basename(path)
    return base.replace('oof_', '').replace('submission_', '')

# Load and align OOFs
oof_dfs = {}
for p in oof_paths:
    t = tag_from(p)
    df = pd.read_csv(p)
    assert {'Id','oof','target'}.issubset(df.columns), f'Bad OOF file cols: {p}'
    oof_dfs[t] = df[['Id','oof','target','fold']].copy()

tags = sorted(oof_dfs.keys())
base = oof_dfs[tags[0]][['Id','target']].rename(columns={'target':'y'})
for t in tags[1:]:
    base = base.merge(oof_dfs[t][['Id','oof']].rename(columns={'oof': f'oof_{t}'}), on='Id', how='inner')
base = base.merge(oof_dfs[tags[0]][['Id','oof']].rename(columns={'oof': f'oof_{tags[0]}'}), on='Id', how='inner')

# Ensure unique column order
pred_cols = [c for c in base.columns if c.startswith('oof_')]
X = base[pred_cols].values.astype(float)
y = base['y'].values.astype(float)

# NNLS weights
w, _ = nnls(X, y)
if w.sum() > 0:
    w = w / w.sum()
weights = dict(zip([c.replace('oof_','') for c in pred_cols], w.tolist()))
blend_oof = X.dot(w)
rmse_blend = mean_squared_error(y, blend_oof, squared=False)
print('NNLS Weights:', weights)
print('Blend OOF RMSE:', rmse_blend)

# Apply weights to SUB files
subs = {}
for p in sub_paths:
    t = tag_from(p)
    subs[t] = pd.read_csv(p)

# Merge predictions on Id
sub_merge = None
used = []
for t in weights.keys():
    if t in subs:
        df = subs[t][['Id','Pawpularity']].rename(columns={'Pawpularity': f'pred_{t}'})
        sub_merge = df if sub_merge is None else sub_merge.merge(df, on='Id', how='inner')
        used.append(t)
print('Using SUB tags:', used)
assert sub_merge is not None and len(used) > 0, 'No matching submissions for weights'

pred = np.zeros(len(sub_merge), dtype=float)
for t in used:
    pred += weights[t] * sub_merge[f'pred_{t}'].clip(0,100).values
final = pd.DataFrame({'Id': sub_merge['Id'], 'Pawpularity': np.clip(pred, 0, 100)})
final.to_csv('submission.csv', index=False)
print('Saved NNLS-blended submission.csv, shape:', final.shape)

Found OOF files: 8
Found SUB files: 8
NNLS Weights: {'convnext_small.in12k_ft_in1k_fusion1_sz384.csv': 0.0, 'convnext_small.in12k_ft_in1k_fusion1_sz384_seed43.csv': 0.12659622936652432, 'convnext_small.in12k_ft_in1k_fusion1_sz512_seed44.csv': 0.1962968613630638, 'convnext_small.in12k_ft_in1k_fusion1_sz512_seed45.csv': 0.1363520627481221, 'convnext_tiny_fusion1_sz384.csv': 0.0898123345552999, 'swin_base_patch4_window12_384_fusion1_sz384_seed42.csv': 0.21454231143062782, 'tf_efficientnetv2_s_in21k_fusion1_sz384.csv': 0.05477408877089537, 'convnext_base_fusion1_sz384_seed42.csv': 0.18162611176546678}
Blend OOF RMSE: 17.370572068035344
Using SUB tags: ['convnext_small.in12k_ft_in1k_fusion1_sz384.csv', 'convnext_small.in12k_ft_in1k_fusion1_sz384_seed43.csv', 'convnext_small.in12k_ft_in1k_fusion1_sz512_seed44.csv', 'convnext_small.in12k_ft_in1k_fusion1_sz512_seed45.csv', 'convnext_tiny_fusion1_sz384.csv', 'swin_base_patch4_window12_384_fusion1_sz384_seed42.csv', 'tf_efficientnetv2_s_in21k_fu

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [24]:
# Smoke test: convnext_base @384, seed=42, CutMix on (fold 0, 3 epochs)
import sys, subprocess, time

cmd = [sys.executable, 'train_cnn.py',
       '--model', 'convnext_base',
       '--img_size', '384',
       '--batch_size', '24',
       '--epochs', '3',
       '--warmup_epochs', '1',
       '--ema_decay', '0.999',
       '--lr_backbone', '2e-5',
       '--lr_head', '8e-4',
       '--weight_decay', '0.05',
       '--num_workers', '4',
       '--seed', '42',
       '--tta_hflip',
       '--use_fusion',
       '--use_cutmix',
       '--cutmix_alpha', '0.5',
       '--cutmix_p', '0.5',
       '--fold', '0'
]
print('Launching ConvNeXt-Base smoke:', ' '.join(cmd), flush=True)
t0 = time.time()
proc = subprocess.run(cmd, check=False)
print('Exit code:', proc.returncode, 'Elapsed:', f'{(time.time()-t0):.1f}s', flush=True)
assert proc.returncode == 0, 'ConvNeXt-Base smoke run failed (check tag/VRAM).'

Launching ConvNeXt-Base smoke: /usr/bin/python3.11 train_cnn.py --model convnext_base --img_size 384 --batch_size 24 --epochs 3 --warmup_epochs 1 --ema_decay 0.999 --lr_backbone 2e-5 --lr_head 8e-4 --weight_decay 0.05 --num_workers 4 --seed 42 --tta_hflip --use_fusion --use_cutmix --cutmix_alpha 0.5 --cutmix_p 0.5 --fold 0


/var/lib/simon/agent_run_states/petfinder-pawpularity-score-20250927-162350/train_cnn.py:353: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device=='cuda'))


Device: cuda
Fold 0 | train=7136 val=1784 epochs=3


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Epoch 1/3 | loss=0.07621 | val_rmse=28.3327 | lr=[2e-05, 0.0008] | time=102.1s


...

Epoch 3/3 | loss=0.02215 | val_rmse=19.9412 | lr=[0.0, 0.0] | time=101.8s
Fold 0 best RMSE: 19.9412 | elapsed 5.1 min | ckpt: work_cnn/convnext_base_seed42_fold0_best.pth


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Saved OOF to work_cnn/oof_convnext_base_fusion1_sz384_seed42.csv OOF RMSE: 39.71039751076835
Saved test predictions to work_cnn/submission_convnext_base_fusion1_sz384_seed42.csv shape: (992, 2)


Exit code: 0 Elapsed: 336.3s


In [25]:
# Full 5-fold: convnext_base @384, seed=42, CutMix on, wd=0.05
import sys, subprocess, time

cmd = [sys.executable, 'train_cnn.py',
       '--model', 'convnext_base',
       '--img_size', '384',
       '--batch_size', '24',
       '--epochs', '12',
       '--warmup_epochs', '1',
       '--ema_decay', '0.999',
       '--lr_backbone', '2e-5',
       '--lr_head', '8e-4',
       '--weight_decay', '0.05',
       '--num_workers', '4',
       '--seed', '42',
       '--tta_hflip',
       '--use_fusion',
       '--use_cutmix',
       '--cutmix_alpha', '0.5',
       '--cutmix_p', '0.5'
]
print('Launching ConvNeXt-Base full run:', ' '.join(cmd), flush=True)
t0 = time.time()
proc = subprocess.run(cmd, check=False)
print('Exit code:', proc.returncode, 'Elapsed:', f'{(time.time()-t0)/60:.1f} min', flush=True)
assert proc.returncode == 0, 'ConvNeXt-Base full run failed'

Launching ConvNeXt-Base full run: /usr/bin/python3.11 train_cnn.py --model convnext_base --img_size 384 --batch_size 24 --epochs 12 --warmup_epochs 1 --ema_decay 0.999 --lr_backbone 2e-5 --lr_head 8e-4 --weight_decay 0.05 --num_workers 4 --seed 42 --tta_hflip --use_fusion --use_cutmix --cutmix_alpha 0.5 --cutmix_p 0.5


/var/lib/simon/agent_run_states/petfinder-pawpularity-score-20250927-162350/train_cnn.py:353: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device=='cuda'))


Device: cuda
Fold 0 | train=7136 val=1784 epochs=12


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Epoch 1/12 | loss=0.07621 | val_rmse=28.3327 | lr=[2e-05, 0.0008] | time=101.4s


...

Epoch 12/12 | loss=0.00206 | val_rmse=17.9220 | lr=[0.0, 0.0] | time=102.1s
Fold 4 best RMSE: 17.8919 | elapsed 20.6 min | ckpt: work_cnn/convnext_base_seed42_fold4_best.pth


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Saved OOF to work_cnn/oof_convnext_base_fusion1_sz384_seed42.csv OOF RMSE: 18.2587059237385
Saved test predictions to work_cnn/submission_convnext_base_fusion1_sz384_seed42.csv shape: (992, 2)


Exit code: 0 Elapsed: 104.2 min


In [27]:
# Launch Run 1 — ConvNeXt-S @512 (fusion), seed 44 per expert advice
import sys, subprocess, time

cmd = [sys.executable, 'train_cnn.py',
       '--model', 'convnext_small.in12k_ft_in1k',
       '--img_size', '512',
       '--batch_size', '24',
       '--epochs', '20',
       '--warmup_epochs', '1',
       '--ema_decay', '0.999',
       '--lr_backbone', '2.5e-5',
       '--lr_head', '8e-4',
       '--weight_decay', '0.05',
       '--num_workers', '4',
       '--seed', '44',
       '--tta_hflip',
       '--use_fusion',
       '--use_cutmix',
       '--cutmix_alpha', '0.5',
       '--cutmix_p', '0.5'
]
print('Launching:', ' '.join(cmd), flush=True)
t0 = time.time()
proc = subprocess.run(cmd, check=False)
print('Exit code:', proc.returncode, 'Elapsed:', f'{(time.time()-t0)/60:.1f} min', flush=True)
assert proc.returncode == 0, 'ConvNeXt-S @512 run failed'

Launching: /usr/bin/python3.11 train_cnn.py --model convnext_small.in12k_ft_in1k --img_size 512 --batch_size 24 --epochs 20 --warmup_epochs 1 --ema_decay 0.999 --lr_backbone 2.5e-5 --lr_head 8e-4 --weight_decay 0.05 --num_workers 4 --seed 44 --tta_hflip --use_fusion --use_cutmix --cutmix_alpha 0.5 --cutmix_p 0.5


/var/lib/simon/agent_run_states/petfinder-pawpularity-score-20250927-162350/train_cnn.py:353: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device=='cuda'))


Device: cuda
Fold 0 | train=7136 val=1784 epochs=20


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Epoch 1/20 | loss=0.08418 | val_rmse=22.1432 | lr=[2.5e-05, 0.0008] | time=122.0s


...

Epoch 20/20 | loss=0.00052 | val_rmse=17.7227 | lr=[0.0, 0.0] | time=121.7s
Fold 4 best RMSE: 17.4273 | elapsed 40.7 min | ckpt: work_cnn/convnext_small.in12k_ft_in1k_seed44_fold4_best.pth


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Saved OOF to work_cnn/oof_convnext_small.in12k_ft_in1k_fusion1_sz512_seed44.csv OOF RMSE: 17.86528552004247
Saved test predictions to work_cnn/submission_convnext_small.in12k_ft_in1k_fusion1_sz512_seed44.csv shape: (992, 2)


Exit code: 0 Elapsed: 205.6 min


In [32]:
# NNLS blend + isotonic calibration on OOF, then apply to test
import os, glob, numpy as np, pandas as pd
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import mean_squared_error
from scipy.optimize import nnls

workdir = 'work_cnn'
oof_paths = sorted(glob.glob(os.path.join(workdir, 'oof_*.csv')))
sub_paths = sorted(glob.glob(os.path.join(workdir, 'submission_*.csv')))
assert oof_paths and sub_paths, 'No artifacts found'

def tag_from(path):
    b = os.path.basename(path)
    return b.replace('oof_', '').replace('submission_', '')

# Load OOFs and align
oof_dfs = {tag_from(p): pd.read_csv(p)[['Id','oof','target']] for p in oof_paths}
tags = sorted(oof_dfs.keys())
base = oof_dfs[tags[0]][['Id','target']].rename(columns={'target':'y'})
for t in tags:
    base = base.merge(oof_dfs[t][['Id','oof']].rename(columns={'oof': f'oof_{t}'}), on='Id', how='inner')

pred_cols = [c for c in base.columns if c.startswith('oof_')]
X = base[pred_cols].values.astype(float)
y = base['y'].values.astype(float)

# NNLS weights (non-neg, sum-to-one)
w, _ = nnls(X, y)
w = w / w.sum() if w.sum() > 0 else w
weights = dict(zip([c.replace('oof_','') for c in pred_cols], w.tolist()))
blend_oof = X.dot(w)
rmse_blend = mean_squared_error(y, blend_oof, squared=False)
print('NNLS OOF RMSE:', rmse_blend, 'weights_n=', len(weights))

# Fit isotonic on OOF
iso = IsotonicRegression(out_of_bounds='clip')
iso.fit(blend_oof, y)
blend_oof_cal = iso.transform(blend_oof)
rmse_cal = mean_squared_error(y, blend_oof_cal, squared=False)
print('Isotonic-calibrated OOF RMSE:', rmse_cal)

# Apply to test
subs = {tag_from(p): pd.read_csv(p)[['Id','Pawpularity']] for p in sub_paths}
sub_merge = None
used = []
for t in weights.keys():
    if t in subs:
        df = subs[t].rename(columns={'Pawpularity': f'pred_{t}'})
        sub_merge = df if sub_merge is None else sub_merge.merge(df, on='Id', how='inner')
        used.append(t)
assert sub_merge is not None and used, 'No matching subs for weights'
pred_test = np.zeros(len(sub_merge), dtype=float)
for t in used:
    pred_test += weights[t] * sub_merge[f'pred_{t}'].clip(0,100).values
pred_test_cal = iso.transform(pred_test)
sub_cal = pd.DataFrame({'Id': sub_merge['Id'], 'Pawpularity': np.clip(pred_test_cal, 0, 100)})
sub_cal.to_csv('submission_calibrated.csv', index=False)
print('Saved submission_calibrated.csv', sub_cal.shape)

NNLS OOF RMSE: 17.437700366849946 weights_n= 7
Isotonic-calibrated OOF RMSE: 17.27066089462739
Saved submission_calibrated.csv (992, 2)


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [30]:
# Launch Run 2 — ConvNeXt-S @512 (fusion), seed 45
import sys, subprocess, time

cmd = [sys.executable, 'train_cnn.py',
       '--model', 'convnext_small.in12k_ft_in1k',
       '--img_size', '512',
       '--batch_size', '24',
       '--epochs', '20',
       '--warmup_epochs', '1',
       '--ema_decay', '0.999',
       '--lr_backbone', '2.5e-5',
       '--lr_head', '8e-4',
       '--weight_decay', '0.05',
       '--num_workers', '4',
       '--seed', '45',
       '--tta_hflip',
       '--use_fusion',
       '--use_cutmix',
       '--cutmix_alpha', '0.5',
       '--cutmix_p', '0.5'
]
print('Launching:', ' '.join(cmd), flush=True)
t0 = time.time()
proc = subprocess.run(cmd, check=False)
print('Exit code:', proc.returncode, 'Elapsed:', f'{(time.time()-t0)/60:.1f} min', flush=True)
assert proc.returncode == 0, 'ConvNeXt-S @512 seed45 run failed'

Launching: /usr/bin/python3.11 train_cnn.py --model convnext_small.in12k_ft_in1k --img_size 512 --batch_size 24 --epochs 20 --warmup_epochs 1 --ema_decay 0.999 --lr_backbone 2.5e-5 --lr_head 8e-4 --weight_decay 0.05 --num_workers 4 --seed 45 --tta_hflip --use_fusion --use_cutmix --cutmix_alpha 0.5 --cutmix_p 0.5


/var/lib/simon/agent_run_states/petfinder-pawpularity-score-20250927-162350/train_cnn.py:353: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device=='cuda'))


Device: cuda
Fold 0 | train=7136 val=1784 epochs=20


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Epoch 1/20 | loss=0.16845 | val_rmse=20.9426 | lr=[2.5e-05, 0.0008] | time=124.2s


...

Epoch 20/20 | loss=0.00046 | val_rmse=17.7177 | lr=[0.0, 0.0] | time=122.0s
Fold 4 best RMSE: 17.4369 | elapsed 40.8 min | ckpt: work_cnn/convnext_small.in12k_ft_in1k_seed45_fold4_best.pth


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Saved OOF to work_cnn/oof_convnext_small.in12k_ft_in1k_fusion1_sz512_seed45.csv OOF RMSE: 17.813752510902557
Saved test predictions to work_cnn/submission_convnext_small.in12k_ft_in1k_fusion1_sz512_seed45.csv shape: (992, 2)


Exit code: 0 Elapsed: 205.5 min


In [34]:
# Launch Run 3 — Swin-Base @384 (fusion), seed 42
import sys, subprocess, time

cmd = [sys.executable, 'train_cnn.py',
       '--model', 'swin_base_patch4_window12_384',
       '--img_size', '384',
       '--batch_size', '24',
       '--epochs', '20',
       '--warmup_epochs', '1',
       '--ema_decay', '0.999',
       '--lr_backbone', '3e-5',
       '--lr_head', '1e-3',
       '--weight_decay', '1e-4',
       '--num_workers', '4',
       '--seed', '42',
       '--tta_hflip',
       '--use_fusion'
]
print('Launching:', ' '.join(cmd), flush=True)
t0 = time.time()
proc = subprocess.run(cmd, check=False)
print('Exit code:', proc.returncode, 'Elapsed:', f'{(time.time()-t0)/60:.1f} min', flush=True)
assert proc.returncode == 0, 'Swin-Base @384 run failed'

Launching: /usr/bin/python3.11 train_cnn.py --model swin_base_patch4_window12_384 --img_size 384 --batch_size 24 --epochs 20 --warmup_epochs 1 --ema_decay 0.999 --lr_backbone 3e-5 --lr_head 1e-3 --weight_decay 1e-4 --num_workers 4 --seed 42 --tta_hflip --use_fusion


/var/lib/simon/agent_run_states/petfinder-pawpularity-score-20250927-162350/train_cnn.py:353: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device=='cuda'))


Device: cuda
Fold 0 | train=7136 val=1784 epochs=20


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Epoch 1/20 | loss=0.04843 | val_rmse=24.7223 | lr=[3e-05, 0.001] | time=160.0s


...

Epoch 20/20 | loss=0.00119 | val_rmse=17.9056 | lr=[0.0, 0.0] | time=162.8s
Fold 4 best RMSE: 17.9056 | elapsed 54.5 min | ckpt: work_cnn/swin_base_patch4_window12_384_seed42_fold4_best.pth


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Saved OOF to work_cnn/oof_swin_base_patch4_window12_384_fusion1_sz384_seed42.csv OOF RMSE: 18.24796057356082
Saved test predictions to work_cnn/submission_swin_base_patch4_window12_384_fusion1_sz384_seed42.csv shape: (992, 2)


Exit code: 0 Elapsed: 274.8 min


In [ ]:
# Launch Run 5 — ConvNeXt-S @512 (image-only), seed 46
import sys, subprocess, time

cmd = [sys.executable, 'train_cnn.py',
       '--model', 'convnext_small.in12k_ft_in1k',
       '--img_size', '512',
       '--batch_size', '24',
       '--epochs', '20',
       '--warmup_epochs', '1',
       '--ema_decay', '0.999',
       '--lr_backbone', '2.5e-5',
       '--lr_head', '8e-4',
       '--weight_decay', '0.05',
       '--num_workers', '4',
       '--seed', '46',
       '--tta_hflip',
       # image-only: do NOT pass --use_fusion
       '--use_cutmix',
       '--cutmix_alpha', '0.5',
       '--cutmix_p', '0.5'
]
print('Launching:', ' '.join(cmd), flush=True)
t0 = time.time()
proc = subprocess.run(cmd, check=False)
print('Exit code:', proc.returncode, 'Elapsed:', f"{(time.time()-t0)/60:.1f} min", flush=True)
assert proc.returncode == 0, 'ConvNeXt-S @512 image-only seed46 run failed'